### Install requirements

In [31]:
# !pip install newspaper3k
# !pip install openai

### Import necessary libraries

In [28]:
import os
from newspaper import Article
import newspaper
import openai
import time
import json
# import tweepy
# from PIL import Image, ImageDraw, ImageFont

#### Collect all URLs from a specified path

In [4]:
def readURLs(filePath):
    '''
    Reads URLs from a given file, where each URL is on a new line,
    and returns a list of URLs.

    Parameters:
    - filePath (str): The path to the text file containing URLs

    Returns:
    - list: A list of URLs
    '''
    # Check if file exists
    if not os.path.exists(filePath):
        print(f'File {filePath} does not exist.')
        return []
    
    # Initialize empty list
    urls = []

    # Open the file and read URLs
    with open(filePath, 'r') as f:
        for line in f:
            # Strip newline characters and add to list
            urls.append(line.strip())
    
    return urls

#### Extract Information from URLS

In [5]:
def extractArticleInfo(urls):
    '''
    Extract titles, authors, and contents of articles from a list of URLs

    Parameters:
    - urls (list): A list of URLs to extract the articles from.

    Returns:
    - list: A list of dictionaries, each containing the title, authors, and content of an article.
    '''
    articlesInfo = []

    for url in urls:
        try:
            # Create Article object
            article = Article(url=url)

            # Downlaod and parse article
            article.download()
            article.parse()

            # Extract information
            info = {
                'title': article.title,
                'author': article.authors[0],
                'content': article.text
            }

            # Append it to the list
            articlesInfo.append(info)
        except newspaper.ArticleException:
            print(f'Failed to retrieve article from {url}')
            continue
        
    return articlesInfo

### Generate Tweets

In [26]:
def generateAlexisPrestonTweets(articles, openaiAPIKey, gptModel):
    '''
    Generates a tweet from Alexis Preston based on the provided article information.

    Parameters:
    - articles (list of dict): A list of dictionaries containing "title", "author", and "content" keys of each article.
    - openaiAPIKey (str): The API key for OpenAI.
    - gptModel (str): The model intended to use for tweet generation. 

    Returns:
    - list of dict: A list of tweets generated by Alexis Preston (title, content, credits).
    '''
    openai.api_key = openaiAPIKey

    personaDescription = """
    Alexis Preston is a 26-year-old tech guru living in New York City with a rich background in software at companies like Google and Apple. 
    She's honest, sometime brutally so, critical, fun, humorous, and occasionally sarcastic, yet highly knowledgeable and loves her fans deeply. Alexis writes comprehensive tech review blogs on Twitter, offering her unique insights on the latest gadgets. Her reviews are known for their depth, wit, and valuable information packed into concise tweets. She owns a skincare business, \'Skin by Alexis\', and while she promotes it, her passion for tech reviews always shines through. Her tweets provide a snapshot review, capturing the essence of the gadgets she discusses. 
    Her tweets are informative, engaging, and reflect her love for the latest tech gadgets while also promoting her skincare brand \' Skin by Alexis \'.
    """

    generatedTweets = []

    for article in articles:

        prompt = f'{personaDescription}\n\n' \
                f'Based on the following article information: \nTitle: {article["title"]}\n' \
                f'Author: {article["author"]}\nContent: {article["content"]}\n' \
                f'generate a Twitter post title, content, and credits that emodies Alexis\'s style and traits.\n' \
                f'The Twitter post\'s title should follow her style and traits from her persona.\nThe Twitter post\'s content will should reflect Alexis\'s personality.\n while demonstrating her deep knowledge of tech. The review should give followers a clear sense of the product\'s pros and cons, and whether Alexis recommends it, all in a single tweet. And lastly, the Twitter post\'s credits would credit herself with a short outro in her style of writing, as well as the original author for inspiration.\n' \
                f'Make sure to return the Twitter post information as a dictionary format with the keys: ["title", "content", "credits"], and don\'t return anything other than the dictionary.'
        
        response = openai.ChatCompletion.create(
            model = gptModel,
            messages = [
                {
                    'role': 'user',
                    'content': prompt
                },
            ],
            temperature = 0.7,
            top_p = 1.0,
            frequency_penalty = 0.5,
            presence_penalty = 0.0
        )

        info = eval(response.choices[0].message.content)

        generatedTweets.append(info)

        time.sleep(10)

    return generatedTweets

### Store Results

In [29]:
def storeResults(generatedTweets, filePath):
    '''
    Stores the generated tweets as a structured JSON file.

    Parameters:
    - generatedTweets (list): A list of dictionaries of all the generated tweets.
    - filePath (str): The file path where the JSON data will be stored.
    '''
    try:
        with open(filePath, 'w', encoding='utf-8') as f:
            json.dump(generatedTweets, f, ensure_ascii=False, indent=4)
        print(f'Data successfully stored in {filePath}')
    except Exception as e:
        print(f'An error occured: {e}')

### Usage

In [30]:
# Specify the path where the 'urls' exist
filePath = 'urls.txt'

# Get a list of all the urls of the articles
urls = readURLs(filePath=filePath)

# Extract information from all the articles, specifically title, author, and content.
articlesInfo = extractArticleInfo(urls=urls)

# Defin OpenAI API Key and Model to use
openaiAPIKey = '<YOUR_API_KEY>'
gptModel = '<GPT MODEL NAME>'

# Generate all the tweets by our persona
generatedTweets = generateAlexisPrestonTweets(
    articles=articlesInfo,
    openaiAPIKey=openaiAPIKey,
    gptModel=gptModel
)

# Save the resultant tweets to an output file
filePath = 'generated_tweets.json'
storeResults(generatedTweets, filePath)

Data successfully stored in generated_tweets.json
